In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [5]:
from scipy.ndimage import maximum_filter

![images](images/Qn1.png)

In [8]:
harris = np.load("harris.npy", allow_pickle=True).item()
Ix_Ix = harris['g*(I_x^2)']
Iy_Iy = harris['g*(I_y^2)']
Ix_Iy = harris['g*(I_x I_y)']
k = 0.06 
tau = 516
Ix_Ix, Iy_Iy, Ix_Iy

(array([[16.8, 18.5, 20. , 20.8, 20.6],
        [21. , 23.4, 25.5, 26.7, 26.5],
        [25.8, 29. , 31.8, 33.4, 33.2],
        [30.4, 34.4, 37.9, 39.9, 39.7],
        [33.9, 38.6, 42.7, 45.1, 44.9]]),
 array([[35.2, 31.8, 27.1, 22. , 17.6],
        [33.3, 30.2, 25.9, 21.4, 17.4],
        [29.3, 26.8, 23.3, 19.5, 16.2],
        [24.4, 22.5, 19.8, 16.9, 14.6],
        [19.5, 18.2, 16.3, 14.4, 12.8]]),
 array([[-6.5, -6.3, -5.2, -3.3, -1. ],
        [-6.7, -6.9, -6. , -4.1, -1.6],
        [-6.5, -7.1, -6.4, -4.7, -2.3],
        [-5.9, -6.7, -6.3, -4.9, -2.8],
        [-4.8, -5.7, -5.7, -4.8, -3.2]]))

In [10]:
structure_tensor = np.asarray([[Ix_Ix, Ix_Iy], [Ix_Iy, Iy_Iy]])
a = structure_tensor[0, 0]
b = structure_tensor[1, 1]
c = structure_tensor[0, 1]
harris_measure = a * b - c**2 - k * (a + b) ** 2

footprint = np.ones((3, 3), dtype=bool)
harris_max = maximum_filter(harris_measure, footprint=footprint, mode="constant", cval=0.0)
local_maxima = (harris_measure == harris_max) & (harris_measure > tau)
corner_coords = np.where(local_maxima)
c = np.column_stack((corner_coords[0], corner_coords[1]))
c


array([[2, 1]])

![images](images/Qn2.png)

## Background context

In [ ]:
def gaussian1DKernel(sigma: int, length: int = 5):
    """
    Generate the 1D gaussian kernel and its derivative
    Args:
        sigma (int): gaussian width
        length (int): length of the Gaussian kernel

    Return:
        g (np.array): gaussian kernel
        gd (np.array): derivative of gaussian kernel, g
    """
    x = np.arange(-np.ceil(length * sigma), np.ceil(length * sigma) + 1)
    constant = 1.0 / (np.sqrt(2 * np.pi) * sigma)
    g = constant * np.exp(-(x**2) / (2 * sigma**2))
    g /= g.sum()
    gd = (-x / sigma**2) * g
    return g, gd


g, gd = gaussian1DKernel(3, 5)
print(g.shape, gd.shape)
plt.plot(g)
plt.plot(gd)

In [ ]:
# # Flip channel due to legacy reasons
# im = cv2.imread(file)[:, :, ::-1]
def gaussianSmoothing(im: np.array, sigma: int):
    """
    Generate the gaussian smoothed image and its smoothed derivative in x and y direction
    Args:
        im (np.array): image to be smoothed
        sigma (int): gaussian width

    Return:
        I (np.array): gaussian smoothed image
        Ix (np.array): smoothed derivative of image im in x direction
        Iy (np.array): smoothed derivative of image im in y direction
    """
    g, gd = gaussian1DKernel(sigma)
    I = cv2.sepFilter2D(im, -1, g, g)
    Ix = cv2.sepFilter2D(im, -1, gd, g)
    Iy = cv2.sepFilter2D(im, -1, g, gd)
    return I, Ix, Iy

In [ ]:
def structureTensor(im: np.array, sigma: int, epsilon: int):
    """
    Generate the structure tensor. We use two Gaussian widths in this function: sigma and epsilon. The first one sigma is used
    to calculate the derivatives and the second one to calculate the structure tensor.
    Args:
        im (np.array): image
        sigma (int): first gaussian width
        epsilon (int): second gaussian width

    Return:
        C (np.array): structure tensor, also referred to as the second-moment matrix, is a matrix derived from the gradient of a function. It describes the distribution of the gradient in a specified neighborhood around a point and makes the information invariant to the observing coordinates.
    """
    _, Ix, Iy = gaussianSmoothing(im, sigma)
    g_eps, _ = gaussian1DKernel(epsilon)
    C = np.asarray(
        [
            [
                cv2.sepFilter2D(Ix**2, -1, g_eps, g_eps),
                cv2.sepFilter2D(Ix * Iy, -1, g_eps, g_eps),
            ],
            [
                cv2.sepFilter2D(Ix * Iy, -1, g_eps, g_eps),
                cv2.sepFilter2D(Iy**2, -1, g_eps, g_eps),
            ],
        ]
    )
    return C

In [ ]:
def harrisMeasure(im: np.array, sigma: int, epsilon: int, k: float):
    """
    Generate a measure how likely a pixel is a corner
    Args:
        im (np.array): image
        sigma (int): first gaussian width
        epsilon (int): second gaussian width
        k (float): free parameter for corner detection

    Return:
        r (np.array): a measure how likely a pixel is a corner
    """
    C = structureTensor(im, sigma, epsilon)
    a = C[0, 0]
    b = C[1, 1]
    c = C[0, 1]
    r = a * b - c**2 - k * (a + b) ** 2
    return r

In [ ]:
def cornerDetector(im: np.array, sigma: int, epsilon: int, k: float, tau: float):
    """
    Detect corners in an image
    Args:
        im (np.array): image to be smoothed
        sigma (int): first gaussian width
        epsilon (int): second gaussian width
        k (float): free parameter for corner detection
        tau (float): relative threshold

    Return:
        c (np.array): list of points where r is the local maximum and larger than tau
    """
    r = harrisMeasure(im, sigma, epsilon, k)
    footprint = np.ones((3, 3), dtype=bool)
    r_max = maximum_filter(r, footprint=footprint, mode="constant", cval=0.0)
    local_maxima = (r == r_max) & (r > tau)
    corner_coords = np.where(local_maxima)
    c = np.column_stack((corner_coords[0], corner_coords[1]))
    return c